In [1]:
import folium
import requests
import pandas
from folium.plugins import HeatMap

In [2]:
df = pandas.read_csv('Crime_Incidents_February_2017_to_Present.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98160 entries, 0 to 98159
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Incident_case_id         98160 non-null  object 
 1   Date                     98160 non-null  object 
 2   Clearance_code_inc_type  98160 non-null  object 
 3   PGPD Reporting Area      98160 non-null  int64  
 4   PGPD Sector              98159 non-null  object 
 5   PGPD Beat                98158 non-null  object 
 6   Street_Number            75906 non-null  object 
 7   Street_Address           98160 non-null  object 
 8   Latitude                 98160 non-null  float64
 9   Longitude                98160 non-null  float64
 10  Location                 98160 non-null  object 
dtypes: float64(2), int64(1), object(8)
memory usage: 8.2+ MB


In [4]:
df.head()

,Incident_case_id,Date,Clearance_code_inc_type,PGPD Reporting Area,PGPD Sector,PGPD Beat,Street_Number,Street_Address,Latitude,Longitude,Location
0,PP20050900000816,05/09/2020,THEFT FROM AUTO,504,TRU,K1,NaN,23RD PKY SB / IVERSON ST WB,38.832853,-76.960127,"(38.8328533917665, -76.9601267725229)"
1,PP17082200000593,08/22/2017,THEFT FROM AUTO,709,E,E4,2100 BLOCK,2143 ROBERT BOWIE DR,38.862053,-76.794117,"(38.8620528131723, -76.7941166162491)"
2,PP19090100001543,09/01/2019,ACCIDENT WITH IMPOUND,710,E,E5,1300 BLOCK,1305 WHISTLING DUCK DR,38.872223,-76.770167,"(38.8722233921289, -76.7701665312052)"
3,PP17062000002184,06/20/2017,ACCIDENT,514,J,J1,4900 BLOCK,4950 INDIAN HEAD HWY NB,38.820386,-76.999825,"(38.8203862309456, -76.9998248666525)"
4,PP19081200001812,08/12/2019,THEFT FROM AUTO,111,A,A6,NaN,25TH AVE / KIRSTON ST,38.986724,-76.967345,"(38.9867244809866, -76.9673446118832)"


In [5]:
# df = df[['Incident_case_id','Date','Clearance_code_inc_type','PGPD Reporting Area','PGPD Sector','PGPD Beat','Street_Number','Street_Address','Latitude','Longitude']]
# df.head()

## Map 1 - Color Coded Crime

In [8]:
# getting a look at the different crimes codes
crimes = sorted(df['Clearance_code_inc_type'].unique())
crimes

['ACCIDENT',
 'ACCIDENT WITH IMPOUND',
 'ASSAULT',
 'ASSAULT, SHOOTING',
 'ASSAULT, WEAPON',
 'AUTO, STOLEN',
 'AUTO, STOLEN & RECOVERED',
 'B & E, COMMERCIAL',
 'B & E, OTHER',
 'B & E, RESIDENTIAL',
 'B & E, RESIDENTIAL (VACANT)',
 'B & E, SCHOOL',
 'B & E, VACANT',
 'HOMICIDE',
 'ROBBERY, COMMERCIAL',
 'ROBBERY, OTHER',
 'ROBBERY, RESIDENTIAL',
 'ROBBERY, VEHICLE',
 'SEX OFFENSE',
 'THEFT',
 'THEFT FROM AUTO',
 'VANDALISM']

In [23]:
crime_cat = set([y[0] for y in [x.split(', ') for x in crimes]])

# df[Crime Label] = df.apply(lambda row: rox['Clearance_code_inc_type'])

{'ACCIDENT',
 'ACCIDENT WITH IMPOUND',
 'ASSAULT',
 'AUTO',
 'B & E',
 'HOMICIDE',
 'ROBBERY',
 'SEX OFFENSE',
 'THEFT',
 'THEFT FROM AUTO',
 'VANDALISM'}

In [ ]:
data = df[df['Clearance_code_inc_type']=='B & E, RESIDENTIAL']

m = folium.Map(location=[38.986, -76.94], zoom_start=14)

HeatMap(
    data=data[['Latitude', 'Longitude']],
    radius=15,
).add_to(m)

m

In [ ]:
data = df[df['Clearance_code_inc_type']=='B & E, RESIDENTIAL']

m = folium.Map(location=[38.986, -76.94], zoom_start=14)

HeatMap(
    data=data[['Latitude', 'Longitude']],
    radius=15,
).add_to(m)

m